In [ ]:
import numpy as np
from scipy.linalg import eigh

# Pauli-Matrizen
I = np.eye(2)
X = np.array([[0, 1], [1, 0]])
Y = np.array([[0, -1j], [1j, 0]])
Z = np.array([[1, 0], [0, -1]])
H_gate = (1 / np.sqrt(2)) * np.array([[1, 1], [1, -1]])

# Tensorprodukt
def tensor(*ops):
    result = ops[0]
    for op in ops[1:]:
        result = np.kron(result, op)
    return result

# Baue Hamiltonian: Z0Z1 + Z1Z2 + X0 + Y1
P1 = tensor(Z, Z, I)
P2 = tensor(I, Z, Z)
P3 = tensor(X, I, I)
P4 = tensor(I, Y, I)
pauli_terms = [P1, P2, P3, P4]
H = sum(pauli_terms)

# Initialzustand: |+++>
zero = np.array([[1], [0]])
psi_000 = tensor(zero, zero, zero)
psi_plus = tensor(H_gate, H_gate, H_gate) @ psi_000

# Krylov-Vektoren erzeugen (Moment k = 0 bis K)
def generate_krylov_vectors(psi, pauli_terms, max_k=1):
    basis_ops = [np.eye(8)]  # Start mit Identität (Moment 0)
    for k in range(1, max_k + 1):
        new_ops = []
        for a in basis_ops:
            for p in pauli_terms:
                op = p @ a
                if not any(np.allclose(op, existing) for existing in basis_ops + new_ops):
                    new_ops.append(op)
        basis_ops.extend(new_ops)
    V = [op @ psi for op in basis_ops]
    return V, basis_ops

# Simulation für gegebenes K
def iqae_run(K):
    V, ops = generate_krylov_vectors(psi_plus, pauli_terms, max_k=K)
    V_matrix = np.column_stack(V)
    Q, R = np.linalg.qr(V_matrix)

    H_reduced = np.array([[np.vdot(Q[:, i], H @ Q[:, j]) for j in range(Q.shape[1])]
                          for i in range(Q.shape[1])])
    eigvals_approx, eigvecs_approx = eigh(H_reduced)
    ground_energy_approx = eigvals_approx[0]
    ground_state_approx = Q @ eigvecs_approx[:, 0]

    eigvals_exact, eigvecs_exact = eigh(H)
    ground_energy_exact = eigvals_exact[0]
    ground_state_exact = eigvecs_exact[:, 0]

    energy_error = np.abs(ground_energy_exact - ground_energy_approx)
    fidelity = np.abs(np.vdot(ground_state_exact, ground_state_approx)) ** 2

    return {
        "K": K,
        "approx_energy": ground_energy_approx,
        "exact_energy": ground_energy_exact,
        "energy_error": energy_error,
        "fidelity": fidelity,
        "dim_reduced": Q.shape[1]
    }

# Beispiel: adaptive Analyse für K = 0 bis 3
results = [iqae_run(K) for K in range(0, 4)]
for r in results:
    print(f"K={r['K']} | approx E={r['approx_energy']:.4f} | error={r['energy_error']:.4f} | fidelity={r['fidelity']:.4f} | dim={r['dim_reduced']}")

K=0 | approx E=1.0000 | error=3.7321 | fidelity=0.0264 | dim=1
K=1 | approx E=-1.9271 | error=0.8049 | fidelity=0.3687 | dim=5
K=2 | approx E=-2.7321 | error=0.0000 | fidelity=0.5000 | dim=8
K=3 | approx E=-2.7321 | error=0.0000 | fidelity=0.5000 | dim=8


In [2]:
eigvals, eigvecs = eigh(H)

# Schwelle für numerische Gleichheit
tol = 1e-8

ground_energy = eigvals[0]
degeneracy = np.sum(np.abs(eigvals - ground_energy) < tol)

print(f"Niedrigster Eigenwert: {ground_energy:.6f}")
print(f"Entartung (Multiplizität): {degeneracy}")

Niedrigster Eigenwert: -2.732051
Entartung (Multiplizität): 2


In [3]:
# Hamiltonian: -Z0Z1 - Z1Z2 - h*(X0 + X1 + X2)
h = 1.0

P1 = -tensor(Z, Z, I)     # -Z0Z1
P2 = -tensor(I, Z, Z)     # -Z1Z2
P3 = -h * tensor(X, I, I) # -h X0
P4 = -h * tensor(I, X, I) # -h X1
P5 = -h * tensor(I, I, X) # -h X2

H = P1 + P2 + P3 + P4 + P5

results = [iqae_run(K) for K in range(0, 4)]
for r in results:
    print(f"K={r['K']} | approx E={r['approx_energy']:.4f} | error={r['energy_error']:.4f} | fidelity={r['fidelity']:.4f} | dim={r['dim_reduced']}")

K=0 | approx E=-3.0000 | error=0.4940 | fidelity=0.8818 | dim=1
K=1 | approx E=-3.4734 | error=0.0205 | fidelity=0.9931 | dim=5
K=2 | approx E=-3.4940 | error=0.0000 | fidelity=1.0000 | dim=8
K=3 | approx E=-3.4940 | error=0.0000 | fidelity=1.0000 | dim=8


In [ ]:
J = -1
Z1Z2 = tensor(Z,Z,I,I,I)